In [1]:
import grequests
from random import randint

### Algorithm for generating random CNPJs

In [14]:
def validate_digit(mult_array, digits_array):
    sum_digits = sum(map(lambda x,y: x*y, mult_array, digits_array))
    dig = sum_digits % 11
    
    if dig < 2:
        dig = 0
    else:
        dig = 11 - dig

    return dig


def generate_cnpj():
    cnpj_12d = str(randint(1000,1e8)).zfill(8) + '0001'
    cnpj_int_l = [int(d) for d in list(cnpj_12d)]

    first_dig = validation_figure([5,4,3,2,9,8,7,6,5,4,3,2], cnpj_int_l)
    sec_dig = validation_figure([6,5,4,3,2,9,8,7,6,5,4,3,2], cnpj_int_l + [first_dig] )
    return cnpj_12d + str(first_dig) + str(sec_dig)


In [15]:
n = 100
list_cnpjs = [generate_cnpj() for i in range(n)]

In [16]:
list_cnpjs

['73524118000192',
 '65900430000105',
 '05066997000104',
 '57789517000129',
 '86901346000186',
 '71419480000178',
 '01900563000107',
 '34648273000169',
 '14722053000120',
 '14448035000100',
 '50407653000168',
 '50689492000142',
 '17328228000108',
 '76288941000134',
 '23714740000195',
 '90139582000184',
 '71443888000185',
 '36647968000151',
 '36065938000137',
 '29541454000151',
 '20082536000100',
 '32119652000136',
 '13522515000101',
 '10831484000174',
 '32869764000104',
 '02602258000193',
 '85645115000196',
 '82295338000182',
 '35167641000110',
 '68164998000101',
 '08338138000107',
 '07792979000127',
 '19189529000123',
 '38879195000164',
 '48548872000115',
 '48088309000101',
 '99653369000160',
 '34632492000150',
 '84055727000166',
 '53896844000129',
 '00613879000147',
 '80020195000170',
 '18119887000105',
 '40245911000101',
 '72275775000180',
 '66351871000168',
 '24412222000180',
 '16551907000189',
 '59380001000151',
 '23523984000190',
 '31424288000155',
 '67174147000179',
 '4190612300